<a href="https://colab.research.google.com/github/aleks-haksly/VIZRO/blob/main/demos/popularity_linechart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vizro -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.0/857.0 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.8/500.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 7.9 MB/s eta 0:00:00


In [2]:
from sqlalchemy import text
from sqlalchemy import create_engine
import pandas as pd
from google.colab import userdata


engine = create_engine(userdata.get("supabase"), client_encoding='utf8', )

def select(sql):
    sql = text(sql)
    return pd.read_sql(sql, engine)

In [3]:
sql = """
WITH t AS (
SELECT
	query,
	date_part('hour',
	ts) AS HOUR,
	count(*),
	ROW_NUMBER() OVER (PARTITION BY
	date_part('hour', ts)
ORDER BY
	count(*) DESC) AS rnb
FROM
	vizro.yandex_data yd
GROUP BY
	query,
	date_part('hour',
	ts))

SELECT platform, date_part('hour', ts) AS hour, query, count(*) FROM vizro.yandex_data yd WHERE query IN (SELECT distinct(query) FROM t WHERE rnb < 6)
GROUP BY platform, date_part('hour', ts),  query
"""
query_df = select(sql)

In [4]:
query_df

,platform,hour,query,count
0,desktop,0.0,porno,14
1,desktop,0.0,sex,12
2,desktop,0.0,xxx,8
3,desktop,0.0,айфон 13,8
4,desktop,0.0,английский алфавит,5
...,...,...,...,...
802,touch,23.0,с днём рождения женщине,46
803,touch,23.0,с днём рождения мужчине,32
804,touch,23.0,секс,312
805,touch,23.0,таблица менделеева,55


In [ ]:
def butterfly_transform_data(df: pd.DataFrame) -> pd.DataFrame:
  query_df = df.copy()
  query_df['date'] = pd.to_datetime(query_df['date'],  format='%y-%m-%d')
  total = query_df.groupby("platform")["date"].count().to_dict()
  filtered = query_df[query_df['query'].isin(set(query_df.groupby(['platform', 'query'], as_index=False)['date'].count().sort_values(by='date',ascending=False).groupby('platform', as_index=False).head(10)["query"]))]
  filtered_group = filtered.groupby(['platform', 'query'])["date"].count().unstack(level=0)
  filtered_group.reset_index(inplace=True)
  for k, v in total.items():
    filtered_group[f'{k}_%'] = filtered_group[k] / v
  return filtered_group

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import vizro.models as vm
from vizro import Vizro
from vizro.models.types import capture


@capture("graph")
def butterfly(data_frame: pd.DataFrame, **kwargs) -> go.Figure:
    fig = px.bar(data_frame, **kwargs)

    orientation = fig.data[0].orientation
    x_or_y = "x" if orientation == "h" else "y"

    fig.update_traces({f"{x_or_y}axis": f"{x_or_y}2"}, selector=1)
    fig.update_layout({f"{x_or_y}axis2": fig.layout[f"{x_or_y}axis"]})
    fig.update_layout(
        {
            f"{x_or_y}axis": {"autorange": "reversed", "domain": [0, 0.5]},
            f"{x_or_y}axis2": {"domain": [0.5, 1]},
        }
    )

    if orientation == "h":
        fig.add_vline(x=0, line_width=2, line_color="grey")
    else:
        fig.add_hline(y=0, line_width=2, line_color="grey")
    fig.data[0].hovertemplate = '<b>%{hovertext}</b><br>% of all=%{x:.2%}<br>qty=%{customdata[0]}<extra></extra>'
    fig.data[0].name = 'desktop'
    fig.data[1].hovertemplate = '<b>%{hovertext}</b><br>% of all=%{x:.2%}<br>qty=%{customdata[1]}<extra></extra>'
    fig.data[1].name = 'touch'
    fig.update_yaxes(categoryorder='min ascending')

    return fig



fig = butterfly(
    butterfly_transform_data(query_df),
    x=["desktop_%", "touch_%"],
    y="query",
    labels={"value": "% of all", "variable": "platform:"},
    hover_name="query", hover_data={'query': False, 'desktop': True, 'touch': True },
)

page = vm.Page(title="My page", components=[vm.Graph(figure=fig)])
dashboard = vm.Dashboard(pages=[page])
Vizro().build(dashboard).run()

<IPython.core.display.Javascript object>